# This is a combination of all the things

### Downloading dependancies

In [1]:
from tweepy import Stream
from tweepy import OAuthHandler
#from tweepy.streaming import StreamListener
import tweepy
import json
import pandas as pd
import csv
import string
import time
import datetime
import numpy as np
from cmath import nan

import numpy as np
import pandas as pd
import os

# Word processing libraries
import re
from nltk.corpus import wordnet
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from shapely.geometry import LineString
from geopandas import GeoDataFrame
from shapely.geometry import Point, Polygon
import folium

### Create Query From Vocab


In [2]:
query_df = pd.read_excel('DATA/Protest-Query.xlsx')

In [3]:
def list_to_or(l):
    out = ''
    for index, word in enumerate(l):
        if word == word:
            out += word + ' OR '

    out = out[:(len(out)-4)]
    return out

In [4]:
uni_kw= list(query_df['University Key Words'])
uni_kw_string = list_to_or(uni_kw)

unis= list(query_df['List of Universities'])
unis_string = list_to_or(unis)

actions= list(query_df['Actions'])
actions_string = list_to_or(actions)

other= list(query_df['Other'])
other_string = list_to_or(other)

In [5]:
query = ('((' + uni_kw_string + ' OR '
            + unis_string + ') ('
            + actions_string + ' OR '
            + other_string + '))')

In [6]:
query += ' -is:retweet lang:en place_country:ZA'

### Creating the Tweet Data. 

In [7]:
# Authenticating Twitter API
# Obtain your Twitter credentials from your twitter developer account

bearer_token = 'AAAAAAAAAAAAAAAAAAAAAEGchQEAAAAAZSYFv1nyLDV81YAKEfDr1fVrlho%3DWKBvyLhQ4CeHrlBRtecAetYkB1ZnAjI3Zydb1516fkIzKhS4vh'
client = tweepy.Client(bearer_token, wait_on_rate_limit=True)

### Search Criteria

In [8]:
search_words = query
date_since = datetime.datetime(2016,1,1)
date_before = datetime.datetime(2018,1,1)
numTweets = 500

In [9]:
# Scrape Tweets
# Note for the final versions we should use -> until_id -> allos to resume scraping from a particualar tweet ID
protest_tweets = []
count = 0 
for response in tweepy.Paginator(client.search_all_tweets, 
                                 query = search_words,
                                 user_fields = ['public_metrics'],
                                 tweet_fields = ['created_at', 'geo', 'public_metrics', 'text'],
                                 place_fields = ['id', 'geo', 'name', 'country_code', 'place_type', 'full_name', 'country'],
                                 expansions = ['author_id', 'geo.place_id'],
                                 start_time = date_since,
                                 end_time = date_before,
                                 max_results=numTweets
                              ):
   time.sleep(1)
   protest_tweets.append(response)

### Populating Data frame

In [10]:
error_count = 0
result = []
user_dict = {}
place_dict = {}
# Loop through each response object
for response in protest_tweets:
    # Take all of the users, and put them into a dictionary of dictionaries with the info we want to keep
    for user in response.includes['users']:
        user_dict[user.id] = {'followers': user.public_metrics['followers_count'],
                              'tweets': user.public_metrics['tweet_count'],
                             }
    
    place_dict = {p['id']: p for p in response.includes['places']}

    for tweet in response.data:
        # For each tweet, find the author's information
        author_info = user_dict[tweet.author_id]
        # Put all of the information we want to keep in a single dictionary for each tweet

        try:
            if place_dict[tweet.geo['place_id']]:
                place = place_dict[tweet.geo['place_id']]   
            else:
                place = nan

            result.append({'tweet_id': tweet.id,
                       'author_followers': author_info['followers'],
                       'text': tweet.text,
                       'created_at': tweet.created_at,
                       'retweets': tweet.public_metrics['retweet_count'],
                       'replies': tweet.public_metrics['reply_count'],
                       'likes': tweet.public_metrics['like_count'],
                       'quote_count': tweet.public_metrics['quote_count'],
                       'place_id': tweet.geo,
                       'place_name': place.full_name,
                       'bbox': place.geo['bbox']

                      })

        except:
            error_count = error_count + 1


print(error_count, " result(s) were excluded due to invalid data points.")        

# Change this list of dictionaries into a dataframe             
df = pd.DataFrame(result) 
tweets = df  

786  result(s) were excluded due to invalid data points.


In [11]:
df.to_csv('DATA/temp.csv')

In [12]:
tweets = pd.read_csv('DATA/temp.csv')
tweets = tweets.drop(['Unnamed: 0'], axis=1)
tweets.head(2)

,tweet_id,author_followers,text,created_at,retweets,replies,likes,quote_count,place_id,place_name,bbox
0,947612173236350976,437807,#KZN We are on thee way! Hitting up Richards B...,2017-12-31 23:35:22+00:00,0,1,21,0,{'place_id': 'd98aef293ae7311d'},"Kempton Park, South Africa","[28.1581593, -26.1689943, 28.3280005, -26.0027..."
1,947609471777374209,1100,@unisa the 3rd January registration is it for ...,2017-12-31 23:24:38+00:00,0,0,0,0,{'place_id': 'b4c5d90b1f1be609'},"Mpumalanga, South Africa","[30.5890085, -29.8534128, 30.7016304, -29.7852..."


In [13]:
tweets.head(1)

,tweet_id,author_followers,text,created_at,retweets,replies,likes,quote_count,place_id,place_name,bbox
0,947612173236350976,437807,#KZN We are on thee way! Hitting up Richards B...,2017-12-31 23:35:22+00:00,0,1,21,0,{'place_id': 'd98aef293ae7311d'},"Kempton Park, South Africa","[28.1581593, -26.1689943, 28.3280005, -26.0027..."


### Adding the previous stuff to it... tweets, location and geolocation

In [14]:
print('Initial size of dataset before dropping duplicated rows:', tweets.shape)
tweets.drop_duplicates(keep = False, inplace = True)

print('Current size of dataset after dropping duplicated rows, if any, is:', tweets.shape)

Initial size of dataset before dropping duplicated rows: (25204, 11)
Current size of dataset after dropping duplicated rows, if any, is: (25204, 11)


In [15]:
tweets.dropna(subset = ['text'], inplace = True)
len(tweets)

25204

### Havent added links and stuff here... We likley wont use in final product. Futhermore text has not been cleaned as this is not needed by either content or sentiment analysis

### Convert Boundry Box to a set of coordinates of Latitude and Longitude
There are a few ways of doing this
* Take an average and find the middle of the Boundry Box
* Indentify where the location is using another API based on place_name
* Keep the location as a polygon and then place each user into a munucipality in hich the area is largest

The way we will do it here is using centroid of the boundry box

### Run from here ======

In [16]:
def bbox_to_coords(bbox):
    all_coords = bbox.split()
    coords = []
    for coordinate in all_coords:
        coordinate = coordinate.replace('[','')
        coordinate = coordinate.replace(']','')
        coordinate = coordinate.replace(',','')
        coord = float(coordinate)
        coords.append(coord)

    return coords

In [17]:
tweets['coords'] = tweets['bbox'].apply(lambda x: bbox_to_coords(x))

In [18]:
tweets['longitude_1'] = tweets['coords'].apply(lambda x: x[0])
tweets['latitude_1'] = tweets['coords'].apply(lambda x: x[1])
tweets['longitude_2'] = tweets['coords'].apply(lambda x: x[2])
tweets['latitude_2'] = tweets['coords'].apply(lambda x: x[3])

In [19]:
tweets = tweets.drop('bbox', axis=1)

In [20]:
def find_centroid(coords):
    geometry = LineString([(coords[0], coords[1]),(coords[2], coords[3])])
    centroid = geometry.centroid
    return centroid

In [21]:
tweets['centroid'] = tweets['coords'].apply(lambda x: find_centroid(x))
tweets['centroid_long'] = tweets['centroid'].apply(lambda z: z.x)
tweets['centroid_lat'] = tweets['centroid'].apply(lambda z: z.y)
tweets = tweets.drop('centroid', axis =1)
tweets.head(3)

,tweet_id,author_followers,text,created_at,retweets,replies,likes,quote_count,place_id,place_name,coords,longitude_1,latitude_1,longitude_2,latitude_2,centroid_long,centroid_lat
0,947612173236350976,437807,#KZN We are on thee way! Hitting up Richards B...,2017-12-31 23:35:22+00:00,0,1,21,0,{'place_id': 'd98aef293ae7311d'},"Kempton Park, South Africa","[28.1581593, -26.1689943, 28.3280005, -26.0027...",28.158159,-26.168994,28.328001,-26.002730,28.243080,-26.085862
1,947609471777374209,1100,@unisa the 3rd January registration is it for ...,2017-12-31 23:24:38+00:00,0,0,0,0,{'place_id': 'b4c5d90b1f1be609'},"Mpumalanga, South Africa","[30.5890085, -29.8534128, 30.7016304, -29.7852...",30.589008,-29.853413,30.701630,-29.785216,30.645319,-29.819315
2,947601603946827776,65,"Can someone help me, when is UJ postgraduate r...",2017-12-31 22:53:22+00:00,1,3,0,0,{'place_id': 'dd9c0d7d7e07eb49'},South Africa,"[16.4475932, -34.8342468, 32.8922934, -22.1247...",16.447593,-34.834247,32.892293,-22.124724,24.669943,-28.479485


### Fix Edge Cases
* Cape Town
* Betty's Bay
* Bloubergstrand
* Mdumbi Beach 
</br>
Using: https://www.distancesto.com/coordinates/za/bloubergstrand-latitude-longitude/history/76385.html

In [22]:
tweets.loc[tweets.place_name == 'Cape Town, South Africa', ['centroid_long', 'centroid_lat']] = 18.4241, -33.9249
tweets.loc[tweets.place_name == 'Mdumbi Beach', ['centroid_long', 'centroid_lat']] = 29.215369, -31.933896
tweets.loc[tweets.place_name == "Betty's Bay, South Africa", ['centroid_long', 'centroid_lat']] = 18.92051, -34.34747
tweets.loc[tweets.place_name == 'Bloubergstrand', ['centroid_long', 'centroid_lat']] = 18.46173, -33.800418

In [23]:
geometry = [Point(xy) for xy in zip(tweets['centroid_long'], tweets['centroid_lat'])]
gdf = GeoDataFrame(tweets, geometry=geometry) 

In [24]:
tweets = tweets.drop('longitude_1', axis=1)
tweets = tweets.drop('longitude_2', axis=1)
tweets = tweets.drop('latitude_1', axis=1)
tweets = tweets.drop('latitude_2', axis=1)
tweets = tweets.drop('centroid_long', axis=1)
tweets = tweets.drop('centroid_lat', axis=1)
tweets.head(2)

,tweet_id,author_followers,text,created_at,retweets,replies,likes,quote_count,place_id,place_name,coords,geometry
0,947612173236350976,437807,#KZN We are on thee way! Hitting up Richards B...,2017-12-31 23:35:22+00:00,0,1,21,0,{'place_id': 'd98aef293ae7311d'},"Kempton Park, South Africa","[28.1581593, -26.1689943, 28.3280005, -26.0027...",POINT (28.24308 -26.08586)
1,947609471777374209,1100,@unisa the 3rd January registration is it for ...,2017-12-31 23:24:38+00:00,0,0,0,0,{'place_id': 'b4c5d90b1f1be609'},"Mpumalanga, South Africa","[30.5890085, -29.8534128, 30.7016304, -29.7852...",POINT (30.64532 -29.81931)


## Removing Useless locations

In [25]:
tweets = tweets[tweets['place_name'] != 'South Africa']
tweets.head(2)

,tweet_id,author_followers,text,created_at,retweets,replies,likes,quote_count,place_id,place_name,coords,geometry
0,947612173236350976,437807,#KZN We are on thee way! Hitting up Richards B...,2017-12-31 23:35:22+00:00,0,1,21,0,{'place_id': 'd98aef293ae7311d'},"Kempton Park, South Africa","[28.1581593, -26.1689943, 28.3280005, -26.0027...",POINT (28.24308 -26.08586)
1,947609471777374209,1100,@unisa the 3rd January registration is it for ...,2017-12-31 23:24:38+00:00,0,0,0,0,{'place_id': 'b4c5d90b1f1be609'},"Mpumalanga, South Africa","[30.5890085, -29.8534128, 30.7016304, -29.7852...",POINT (30.64532 -29.81931)


# Adding location based on shape file

In [26]:
#Nan
from cmath import nan


import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
from shapely.geometry import Point, Polygon
import folium
from shapely.geometry import LineString
from sympy import centroid
from geopandas import GeoDataFrame
# importing geopy library
from geopy.geocoders import Nominatim
from numpy import NaN
from shapely import wkt

# importing all the dependancies
import numpy as np
import pandas as pd 
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
from datetime import datetime
from datetime import timedelta

import pandas as pd
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

### Downloading Tweets

### Converting created at to a datetime object

In [27]:
from datetime import datetime

dates = tweets['created_at']
datesOBJ =[]
for date in dates:
    date_object = datetime.fromisoformat(date)
    date_object = date_object.date()
    datesOBJ.append(date_object)

tweets['Date'] = datesOBJ

### Downloading Tweets

In [28]:
tweets = tweets[['text','Date','geometry','author_followers','retweets','replies','likes','quote_count']]

In [29]:
tweets.head(2)

,text,Date,geometry,author_followers,retweets,replies,likes,quote_count
0,#KZN We are on thee way! Hitting up Richards B...,2017-12-31,POINT (28.24308 -26.08586),437807,0,1,21,0
1,@unisa the 3rd January registration is it for ...,2017-12-31,POINT (30.64532 -29.81931),1100,0,0,0,0


# Adding the sentiment

It would be way better if i made this a pandas function as opposed to creating a separte list. 

In [30]:
# creating a dataframe with the text as the list
text = tweets['text'].tolist()
text = pd.DataFrame(text, columns = ['Text'])

In [31]:
# this is a cleaning text function... very simular to the one in tweet_cleaning. May just combine this into one function in tweet cleaning. 
def cleanTxt(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # removes mentions
    text = re.sub(r'#', '', text) # removes #
    text = re.sub(r'RT[\s]+', '', text) # removes RT
    text = re.sub(r'https?:\/\/\S+', '', text) # removes hyper link

    return text
text['Text']=text['Text'].apply(cleanTxt)

In [32]:
# get the subjectivity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

# get the polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

# function to compute the negative, neutral and positive analysis
def getAnalysis(score):
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Positive'
    
text['Subjectivity'] = text['Text'].apply(getSubjectivity)
text['Polarity'] = text['Text'].apply(getPolarity)
text['Analysis'] = text['Polarity'].apply(getAnalysis)


text = text.drop(['Text'], axis = 1)

In [33]:
# Adding the subjectivity to the tweets
tweets = pd.concat([tweets, text.reindex(tweets.index)], axis = 1)

In [34]:

tweets.to_csv('DATA/rawtweets.csv')